<h1 style="color:#3da1da;font-size: 300%;" id="timeshap tutorial" align="center"  >TimeSHAP Tutorial - TensorFlow - AReM dataset</h1><p>&nbsp;

<a id='top_cell'></a>

## Table of contents
1. [Data Processing](#1.-Data-Processing)
  1. [Data Loading](#1.1-Data-Loading)
  2. [Data Treatment](#1.2-Data-Treatment)
2. [Model](#2.-Model)
  1. [Model Definition](#2.1-Model-Definition)
  2. [Model Training](#2.2-Model-Training)
3. [TimeSHAP](#3.-TimeSHAP)
  1. [Local Explanations](#3.1-Local-Explanations)
  2. [Global Explanations](#3.2-Global-Explanations)
  3. [Individual Plots](#3.3-Individual-Plots)
    

# TimeSHAP

TimeSHAP is a model-agnostic, recurrent explainer that builds upon KernelSHAP and extends it to the sequential domain. 

TimeSHAP computes local event/timestamp- feature-, and cell-level attributions. 
    
Aditionally TimeSHAP also computes global event- and feature-level explanations.
    
As sequences can be arbitrarily long, TimeSHAP also implements a pruning algorithm based on Shapley Values, 
that finds a subset of consecutive, recent events that contribute the most to the decision.

---
# 1. Data-Processing
---

In [60]:
import pandas as pd
import numpy as np
import os
import re

np.random.seed(42)

import warnings
warnings.filterwarnings('ignore')

from timeshap import __version__
__version__

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## 1.2 Data Treatment

### Separate in train and test

In [61]:
# choose ids to use for test
df = pd.read_csv('Stocks\IBM_tshap.csv')
df.rename(columns = {'Adj Close':'AdjClose'}, inplace = True)
step_size = 5

In [62]:
df.head(10)

,Date,Open,High,Low,Close,AdjClose,Volume,seq_id,tClose,timestep
0,1/2/1962,7.374124,7.374124,7.291268,7.291268,1.526556,407940,1,6.985341,1
1,1/3/1962,7.291268,7.355003,7.291268,7.355003,1.539900,305955,1,6.985341,2
2,1/4/1962,7.355003,7.355003,7.278521,7.281708,1.524555,274575,1,6.985341,3
3,1/5/1962,7.272148,7.272148,7.125558,7.138305,1.494532,384405,1,6.985341,4
4,1/8/1962,7.131931,7.131931,6.947100,7.004461,1.466509,572685,1,6.985341,5
5,1/9/1962,7.036329,7.176546,7.036329,7.087317,1.483856,517770,1,6.985341,6
6,1/10/1962,7.100064,7.131931,7.100064,7.100064,1.486523,313800,1,6.985341,7
7,1/11/1962,7.119184,7.176546,7.119184,7.176546,1.502537,337335,1,6.985341,8
8,1/12/1962,7.189293,7.240280,7.189293,7.189293,1.505206,462855,1,6.985341,9
9,1/15/1962,7.214786,7.237094,7.214786,7.221160,1.511879,266730,1,6.985341,10


In [63]:
df = df[['Open','Close','High','Low','AdjClose','Volume','timestep','seq_id','tClose']]

In [64]:
train_sequence_end = int(0.8*len(df)/step_size)

In [65]:
df_len = len(df)
train_len = train_sequence_end*step_size
d_train =  df[:train_len]
d_test = df[train_len:]

In [66]:
print(len(d_train))
print(len(d_test))

370295
92575


###  Normalize Features

In [67]:
class NumericalNormalizer:
    def __init__(self, fields: list):
        self.metrics = {}
        self.fields = fields

    def fit(self, df: pd.DataFrame ) -> list:
        means = df[self.fields].mean()
        std = df[self.fields].std()
        for field in self.fields:
            field_mean = means[field]
            field_stddev = std[field]
            self.metrics[field] = {'mean': field_mean, 'std': field_stddev}

    def transform(self, df: pd.DataFrame) -> pd.DataFrame:
        # Transform to zero-mean and unit variance.
        for field in self.fields:
            f_mean = self.metrics[field]['mean']
            f_stddev = self.metrics[field]['std']
            # OUTLIER CLIPPING to [avg-3*std, avg+3*avg]
            df[field] = df[field].apply(lambda x: f_mean - 3 * f_stddev if x < f_mean - 3 * f_stddev else x)
            df[field] = df[field].apply(lambda x: f_mean + 3 * f_stddev if x > f_mean + 3 * f_stddev else x)
            if f_stddev > 1e-5:
                df[f'p_{field}_normalized'] = df[field].apply(lambda x: ((x - f_mean)/f_stddev))
            else:
                df[f'p_{field}_normalized'] = df[field].apply(lambda x: x * 0)
        return df

In [68]:
raw_model_features = ['Open','Close','High','Low','AdjClose', 'Volume']
#all features are numerical
raw_model_features.append('tClose')
normalizor = NumericalNormalizer(raw_model_features)
normalizor.fit(d_train)
d_train_normalized = normalizor.transform(d_train)
d_test_normalized = normalizor.transform(d_test)

raw_model_features.pop()

'tClose'

In [69]:
d_train_normalized

,Open,Close,High,Low,AdjClose,Volume,timestep,seq_id,tClose,p_Open_normalized,p_Close_normalized,p_High_normalized,p_Low_normalized,p_AdjClose_normalized,p_Volume_normalized,p_tClose_normalized
0,7.374124,7.291268,7.374124,7.291268,1.526556,407940.0,1,1,6.985341,-0.886727,-0.888675,-0.887805,-0.887675,-0.801703,-0.951766,-0.896955
1,7.291268,7.355003,7.355003,7.291268,1.539900,305955.0,2,1,6.985341,-0.889058,-0.886884,-0.888336,-0.887675,-0.801093,-0.972215,-0.896955
2,7.355003,7.281708,7.355003,7.278521,1.524555,274575.0,3,1,6.985341,-0.887265,-0.888944,-0.888336,-0.888037,-0.801794,-0.978507,-0.896955
3,7.272148,7.138305,7.272148,7.125558,1.494532,384405.0,4,1,6.985341,-0.889596,-0.892974,-0.890639,-0.892388,-0.803166,-0.956485,-0.896955
4,7.131931,7.004461,7.131931,6.947100,1.466509,572685.0,5,1,6.985341,-0.893540,-0.896735,-0.894536,-0.897463,-0.804447,-0.918733,-0.896955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370290,141.481842,142.543015,142.724670,141.175903,84.717658,4279709.0,1,12344,146.509027,2.885665,2.912348,2.874020,2.920100,3.000000,-0.175441,3.000000
370291,142.676865,142.275330,142.724670,141.730408,84.717658,3604307.0,2,12344,146.509027,2.919281,2.904826,2.874020,2.935871,3.000000,-0.310865,3.000000
370292,142.342255,143.403442,143.403442,141.940720,84.717658,4753233.0,3,12344,146.509027,2.909868,2.936529,2.892885,2.941852,3.000000,-0.080495,3.000000
370293,143.231354,144.024857,144.799240,142.810699,84.717658,9599037.0,4,12344,146.509027,2.934878,2.953993,2.931679,2.966595,3.000000,0.891132,3.000000


### Features

In [70]:
model_features = [f"p_{x}_normalized" for x in raw_model_features]
time_feat = 'timestep'
label_feat = 'p_tClose_normalized'
sequence_id_feat = 'seq_id'

plot_feats = {
    'p_Open_normalized': "Open",
    'p_Close_normalized': "Close",
    'p_High_normalized': "High",
    'p_Low_normalized': "Low",
    'p_AdjClose_normalized': "AdjClose",
    'p_Volume_normalized': "Volume",
}

In [71]:
print(len(d_train_normalized))

370295


This example notebook requires TensorFlow!

Install it if you haven't already:
```
!pip install tensorflow
```

In [72]:
# df_3 = pd.read_csv('Stocks\IBM.csv')
# df_3 = df_3[-2000:]
# df_3.reset_index()
# df_3 = df_3[['Open','Close', 'High', 'Low','Adj Close', 'Volume']]
# df_3.rename(columns = {'Adj Close':'AdjClose'}, inplace = True)

# df_3_len = len(df_3)
# split_len = int(0.8*df_3_len)
# df_3_train =  df_3[:split_len]
# df_3_test = df_3[split_len:]

# raw_model_features = ['Open','Close','High','Low','AdjClose', 'Volume']
# #all features are numerical
# raw_model_features
# normalizor = NumericalNormalizer(raw_model_features)
# normalizor.fit(df_3_train)
# df_3_train_normalized = normalizor.transform(df_3_train)
# df_3_test_normalized = normalizor.transform(df_3_test)

# df_3_train_normalized.reset_index(drop=True, inplace=True)
# df_3_test_normalized.reset_index(drop=True, inplace=True)

# df_3_test_normalized = df_3_test_normalized[['p_Open_normalized','p_Close_normalized','p_High_normalized','p_Low_normalized','p_AdjClose_normalized','p_Volume_normalized']]
# df_3_train_normalized = df_3_train_normalized[['p_Open_normalized','p_Close_normalized','p_High_normalized','p_Low_normalized','p_AdjClose_normalized','p_Volume_normalized']]

In [73]:
def df_to_numpy(df, model_feats, label_feat, group_by_feat, timestamp_Feat):
    sequence_length = len(df[timestamp_Feat].unique())

    data_tensor = np.zeros(
        (len(df[group_by_feat].unique()), sequence_length, len(model_feats)))
    labels_tensor = np.zeros((len(df[group_by_feat].unique()), 1))

    for i, name in enumerate(df[group_by_feat].unique()):
        name_data = df[df[group_by_feat] == name]
        sorted_data = name_data.sort_values(timestamp_Feat)

        data_x = sorted_data[model_feats].values
        labels = sorted_data[label_feat].values
        #assert labels.sum() == 0 or labels.sum() == len(labels)
        data_tensor[i, :, :] = data_x
        labels_tensor[i, :] = labels[0]
    return data_tensor, labels_tensor

In [74]:
X_train, y_train = df_to_numpy(d_train_normalized, model_features, label_feat, sequence_id_feat, time_feat)

X_test, y_test = df_to_numpy(d_test_normalized, model_features, label_feat, sequence_id_feat, time_feat)

ValueError: could not broadcast input array from shape (5,6) into shape (30,6)

In [ ]:

print("Training input shape: ", X_train.shape)
print("Training input shape: ", y_train.shape)

print("Test input shape: ", X_test.shape)
print("Test input shape: ", y_test.shape)

Training input shape:  (1596, 5, 6)
Training input shape:  (1596, 1)
Test input shape:  (399, 5, 6)
Test input shape:  (399, 1)


In [ ]:
# def generate_sequences(data, window_size):
#   _l = len(data) 
#   Xs = []
#   Ys = []
#   for i in range(0, (_l - window_size)):
#     Xs.append(np.array(data[i:i+window_size]))
#     #print(data[i+window_size,:])
#     Ys.append([data['p_Close_normalized'][i+window_size]])
#   return np.array(Xs), np.array(Ys)

# TIME_STEPS = 5
# X_train, y_train= generate_sequences(df_3_train_normalized, TIME_STEPS)
# print("Training input shape: ", X_train.shape)
# print("Training input shape: ", y_train.shape)

# X_test, y_test = generate_sequences(df_3_test_normalized, TIME_STEPS)

# print("Test input shape: ", X_test.shape)
# print("Test input shape: ", y_test.shape)

___
# 2. Model


This example notebook requires Tensorflow!

Install it if you haven't already:
```
!pip install tensorflow
```

## 2.1 Model Definition

In [ ]:
import tensorflow as tf

inputs = tf.keras.layers.Input(shape=(None, 6))
lstm1 = tf.keras.layers.LSTM(64)(inputs)
ff1 = tf.keras.layers.Dense(64, activation='relu')(lstm1)
ff2 = tf.keras.layers.Dense(1)(ff1)
model = tf.keras.models.Model(inputs=inputs, outputs=ff2)

## 2.2 Model Training

In [ ]:
model.compile(loss='mse',
              optimizer=tf.keras.optimizers.Adam(0.001))

model.fit(X_train, y_train, epochs=20, batch_size=55, validation_data=(X_test, y_test))

Epoch 1/20
30/30 [==============================] - 5s 34ms/step - loss: 0.3529 - val_loss: 0.0581
Epoch 2/20
30/30 [==============================] - 0s 6ms/step - loss: 0.0459 - val_loss: 0.0446
Epoch 3/20
30/30 [==============================] - 0s 6ms/step - loss: 0.0367 - val_loss: 0.0387
Epoch 4/20
30/30 [==============================] - 0s 6ms/step - loss: 0.0338 - val_loss: 0.0375
Epoch 5/20
30/30 [==============================] - 0s 6ms/step - loss: 0.0321 - val_loss: 0.0375
Epoch 6/20
30/30 [==============================] - 0s 6ms/step - loss: 0.0306 - val_loss: 0.0345
Epoch 7/20
30/30 [==============================] - 0s 6ms/step - loss: 0.0292 - val_loss: 0.0360
Epoch 8/20
30/30 [==============================] - 0s 6ms/step - loss: 0.0278 - val_loss: 0.0345
Epoch 9/20
30/30 [==============================] - 0s 16ms/step - loss: 0.0287 - val_loss: 0.0293
Epoch 10/20
30/30 [==============================] - 0s 10ms/step - loss: 0.0260 - val_loss: 0.0295
Epoch 11/20
30/3

---
# 3. TimeSHAP
---

### Model entry point

In [ ]:
import timeshap
f = lambda x: model.predict(x)

In [ ]:
d_train_normalized.head()

,Open,Close,High,Low,AdjClose,Volume,timestep,seq_id,tClose,p_Open_normalized,p_Close_normalized,p_High_normalized,p_Low_normalized,p_AdjClose_normalized,p_Volume_normalized,p_tClose_normalized
0,160.277252,160.353729,160.936905,159.847031,107.469635,2443247.0,1,1,155.803055,1.678193,1.686197,1.670350,1.706619,0.432944,-0.820430,1.377144
1,160.975143,161.204590,162.495224,160.200760,108.039925,3871874.0,2,1,155.803055,1.726602,1.745277,1.779722,1.730924,0.494842,-0.343851,1.377144
2,161.290634,159.627151,161.414917,159.493301,106.982735,3711313.0,3,1,155.803055,1.748485,1.635746,1.703900,1.682315,0.380097,-0.397413,1.377144
3,160.420654,158.776291,160.736145,158.776291,106.412460,2519709.0,4,1,155.803055,1.688140,1.576666,1.656259,1.633050,0.318200,-0.794923,1.377144
4,158.919693,158.183563,159.474182,157.963669,106.015198,9357516.0,5,1,155.803055,1.584026,1.535510,1.567687,1.577215,0.275082,1.486119,1.377144


### Baseline event

In [ ]:
from timeshap.utils import calc_avg_event
average_event = calc_avg_event(d_train_normalized, numerical_feats=model_features, categorical_feats=[])

In [ ]:
average_event

,p_Open_normalized,p_Close_normalized,p_High_normalized,p_Low_normalized,p_AdjClose_normalized,p_Volume_normalized
0,0.043552,0.044569,0.036815,0.048997,0.007837,-0.258937


### Baseline event

In [ ]:
sequence_id_feat

'seq_id'

In [ ]:
d_train_normalized.head()

,Open,Close,High,Low,AdjClose,Volume,timestep,seq_id,tClose,p_Open_normalized,p_Close_normalized,p_High_normalized,p_Low_normalized,p_AdjClose_normalized,p_Volume_normalized,p_tClose_normalized
0,160.277252,160.353729,160.936905,159.847031,107.469635,2443247.0,1,1,155.803055,1.678193,1.686197,1.670350,1.706619,0.432944,-0.820430,1.377144
1,160.975143,161.204590,162.495224,160.200760,108.039925,3871874.0,2,1,155.803055,1.726602,1.745277,1.779722,1.730924,0.494842,-0.343851,1.377144
2,161.290634,159.627151,161.414917,159.493301,106.982735,3711313.0,3,1,155.803055,1.748485,1.635746,1.703900,1.682315,0.380097,-0.397413,1.377144
3,160.420654,158.776291,160.736145,158.776291,106.412460,2519709.0,4,1,155.803055,1.688140,1.576666,1.656259,1.633050,0.318200,-0.794923,1.377144
4,158.919693,158.183563,159.474182,157.963669,106.015198,9357516.0,5,1,155.803055,1.584026,1.535510,1.567687,1.577215,0.275082,1.486119,1.377144


In [ ]:
from timeshap.utils import calc_avg_sequence
average_sequence = calc_avg_sequence(d_train_normalized, numerical_feats=model_features, categorical_feats=[],model_features=model_features, entity_col=sequence_id_feat)

In [ ]:
print(average_sequence)
print(average_sequence.shape)

[[1.36711288e+02 1.36720848e+02 1.37705551e+02 1.35755264e+02
  1.03552960e+02 4.12223350e+06]
 [1.36711288e+02 1.36711288e+02 1.37686424e+02 1.35745704e+02
  1.03552960e+02 4.12448250e+06]
 [1.36711288e+02 1.36711288e+02 1.37662521e+02 1.35721801e+02
  1.03552960e+02 4.12641750e+06]
 [1.36711288e+02 1.36711288e+02 1.37657745e+02 1.35707458e+02
  1.03552960e+02 4.13143850e+06]
 [1.36701721e+02 1.36706504e+02 1.37648186e+02 1.35702675e+02
  1.03552960e+02 4.13588400e+06]]
(5, 6)


### Average score over baseline

In [ ]:
from timeshap.utils import get_avg_score_with_avg_event
avg_score_over_len = get_avg_score_with_avg_event(f, average_event, top=480)

1/1 [==============================] - 0s 39ms/step


## 3.1 Local Explanations

### Select sequences to explain

In [ ]:
data_point = 300

x_data = X_test[data_point].reshape(1,X_test.shape[1],X_test.shape[2])
sequence_id = data_point+1

### Local Report on positive instance

In [ ]:
from timeshap.explainer import local_report

pruning_dict = {'tol': 0.025}
event_dict = {'rs': 42, 'nsamples': 32000}
feature_dict = {'rs': 42, 'nsamples': 32000, 'feature_names': model_features, 'plot_features': plot_feats}
cell_dict = {'rs': 42, 'nsamples': 32000, 'top_x_feats': 2, 'top_x_events': 2}
local_report(f, x_data, pruning_dict, event_dict, feature_dict, cell_dict=cell_dict, entity_uuid=sequence_id, entity_col='seq_id', baseline=average_event)

Assuming all features are model features
16/16 [==============================] - 0s 3ms/step


alt.HConcatChart(...)

In [ ]:
x_pd = d_test_normalized[d_test_normalized['seq_id'] == train_sequence_end+data_point+1]
x_pd

,Open,Close,High,Low,AdjClose,Volume,timestep,seq_id,tClose,p_Open_normalized,p_Close_normalized,p_High_normalized,p_Low_normalized,p_AdjClose_normalized,p_Volume_normalized,p_tClose_normalized
9480,140.539993,140.889999,140.899994,139.449997,131.120914,2858000.0,1,1897,143.550003,0.309128,0.334720,0.264040,0.305158,3.0,-0.682072,0.525128
9481,141.100006,141.550003,141.899994,140.479996,131.120914,3338600.0,2,1897,143.550003,0.347973,0.380548,0.334226,0.375928,3.0,-0.521747,0.525128
9482,142.070007,142.600006,143.619995,141.369995,131.120914,3869200.0,3,1897,143.550003,0.415257,0.453456,0.454946,0.437079,3.0,-0.344743,0.525128
9483,142.440002,141.110001,142.500000,140.009995,131.120914,2866600.0,4,1897,143.550003,0.440922,0.349996,0.376338,0.343635,3.0,-0.679203,0.525128
9484,142.380005,143.699997,144.250000,141.580002,131.120914,3574000.0,5,1897,143.550003,0.436760,0.529834,0.499163,0.451509,3.0,-0.443219,0.525128


## 3.2 Global Explanations

### Explain all 

TimeSHAP offers methods to explain all instances and save as CSV.
This allows for global explanations and local plots with no calculation delay.

In [ ]:
len(d_test_normalized)

1995

In [ ]:
from timeshap.explainer import global_report

#pos_dataset = d_test_normalized[d_test_normalized[label_feat] > 120]
pos_dataset = d_test_normalized[1000:]
schema = list(pos_dataset.columns)
pruning_dict = {'tol': [0.05, 0.075], 'path': 'outputs/prun_all_tf.csv'}
event_dict = {'path': 'outputs/event_all_tf.csv', 'rs': 42, 'nsamples': 32000}
feature_dict = {'path': 'outputs/feature_all_tf.csv', 'rs': 42, 'nsamples': 32000, 'feature_names': model_features, 'plot_features': plot_feats}


In [ ]:
prun_stats, global_plot = global_report(f, pos_dataset, pruning_dict, event_dict, feature_dict, average_event, model_features, schema, sequence_id_feat, time_feat)
prun_stats

The defined path for pruning data already exists and the append option is turned off. TimeSHAP will only read from this file and will not create new explanation data
The defined path for event explanations already exists and the append option is turned off. TimeSHAP will only read from this file and will not create new explanation data
Calculating pruning algorithm
Calculating event data
Calculating feat data
2/2 [==============================] - 0s 6ms/step


AssertionError: Pruning idx must be smaller than the sequence length. If not all events are pruned

In [ ]:
global_plot

alt.VConcatChart(...)

## 3.3 Individual Plots

### Local Plots

In [ ]:
from timeshap.plot import plot_temp_coalition_pruning, plot_event_heatmap, plot_feat_barplot, plot_cell_level
from timeshap.explainer import local_pruning, local_event, local_feat, local_cell_level
# select model features only

x_data = x_pd[model_features]
# convert the instance to numpy so TimeSHAP receives it
x_data = np.expand_dims(x_data.to_numpy().copy(), axis=0)

NameError: name 'pos_x_pd' is not defined

##### Pruning algorithm

In [ ]:
pruning_dict = {'tol': 0.025,}
coal_plot_data, coal_prun_idx = local_pruning(f, x_data, pruning_dict, average_event, positive_sequence_id, sequence_id_feat, False)
# coal_prun_idx is in negative terms
pruning_idx = pos_x_data.shape[1] + coal_prun_idx
pruning_plot = plot_temp_coalition_pruning(coal_plot_data, coal_prun_idx, plot_limit=40)
pruning_plot

##### Event-level explanation

In [ ]:
event_dict = {'rs': 42, 'nsamples': 32000}
event_data = local_event(f, pos_x_data, event_dict, positive_sequence_id, sequence_id_feat, average_event, pruning_idx)
event_plot = plot_event_heatmap(event_data)
event_plot

In [ ]:
event_plot

##### Feature-level explanation

In [ ]:
feature_dict = {'rs': 42, 'nsamples': 32000, 'feature_names': model_features, 'plot_features': plot_feats}
feature_data = local_feat(f, pos_x_data, feature_dict, positive_sequence_id, sequence_id_feat, average_event, pruning_idx)
feature_plot = plot_feat_barplot(feature_data, feature_dict.get('top_feats'), feature_dict.get('plot_features'))
feature_plot

##### Cell-level explanation

In [ ]:
cell_dict = {'rs': 42, 'nsamples': 32000, 'top_x_events': 3, 'top_x_feats': 3}
cell_data = local_cell_level(f, pos_x_data, cell_dict, event_data, feature_data, positive_sequence_id, sequence_id_feat, average_event, pruning_idx)
feat_names = list(feature_data['Feature'].values)[:-1] # exclude pruned events
cell_plot = plot_cell_level(cell_data, feat_names, feature_dict.get('plot_features'))
cell_plot

### Global Plots

In [ ]:
from timeshap.explainer import prune_all, pruning_statistics, event_explain_all, feat_explain_all
from timeshap.plot import plot_global_event, plot_global_feat

pos_dataset = d_test_normalized[d_test_normalized['label'] == 1]

##### Pruning statistics

In [ ]:
pruning_dict = {'tol': [0.05, 0.075], 'path': 'outputs/prun_all_tf.csv'}
prun_indexes = prune_all(f, pos_dataset, pruning_dict, average_event, model_features, schema, sequence_id_feat, time_feat)
pruning_stats = pruning_statistics(prun_indexes, pruning_dict.get('tol'))
pruning_stats

##### Global event-level

In [ ]:
event_dict = {'path': 'outputs/event_all_tf.csv', 'rs': 42, 'nsamples': 32000}
event_data = event_explain_all(f, pos_dataset, event_dict, prun_indexes, average_event, model_features, schema, sequence_id_feat, time_feat)
event_global_plot = plot_global_event(event_data)
event_global_plot

##### Global feature-level

In [ ]:
feature_dict = {'path': 'outputs/feature_all_tf.csv', 'rs': 42, 'nsamples': 32000, 'feature_names': model_features, 'plot_features': plot_feats, }
feat_data = feat_explain_all(f, pos_dataset, feature_dict, prun_indexes, average_event, model_features, schema, sequence_id_feat, time_feat)
feat_global_plot = plot_global_feat(feat_data, **feature_dict)
feat_global_plot